In [1]:
from hypergraph import HyperGraph

h = HyperGraph.fromHyperbench("/home/okulmus/Documents/OldBenchmarks/benchmark/hyperbench/1.dtl")

In [2]:
def all_choose_k(S, k):
    from itertools import chain, combinations
    return chain(*(combinations(S, kp) for kp in range(1,k+1)))
#
def all_lambdas(E, k):
    for es in all_choose_k(E, k):
        yield set.union(*es)

In [7]:
%reload_ext line_profiler
%load_ext autoreload
%autoreload 2

from functools import partial
from ctdcheck import Block
from ctdcheck import VertSet
from hypergraph import HyperGraph

# (over approximates) the bags produced by the LogK algorithm
def computesoftk(h, k):
    softk = list()
    for P in all_lambdas(h.E,k):
        for C in h.separate(P, only_vertices=True):
            for L in all_lambdas(h.E, k):
                B = set.intersection(C, L)
                if len(B) > 1 and B not in softk:
                    softk.append(B)
    return softk
    
# computes the blocks of a bag by computing its components w.r.t. h
def bag_to_blocks(h,B):  
    blocks = list()
    for C in h.separate(B, only_vertices=True):
        blocks.append(Block(VertSet(B),VertSet(C)))
    blocks.append(Block(VertSet(B),VertSet({})))  # adding trivial block too
    return blocks

# Same as  computeosftK, but returns directly the blocks
def computesoftkBlocks(h, k):
    out = list()
    listOfLists = map(partial(bag_to_blocks,h),computesoftk(h,k))
    for ll in listOfLists:
        for l in ll:
            out.append(l)
    return out




from glob import glob

for path in glob("../hyperbench/1.dtl"):
    h = HyperGraph.fromHyperbench(path)
    #print(len(h.V), len(h.E))
    _ = computesoftk(h, 3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%lprun -f computesoftk computesoftk(h,3)

Timer unit: 1e-09 s

Total time: 0.300876 s
File: /tmp/ipykernel_336005/3625937043.py
Function: computesoftk at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def computesoftk(h, k):
    12         1       1066.0   1066.0      0.0      softk = list()
    13       378     235353.0    622.6      0.1      for P in all_lambdas(h.E,k):
    14      1214   18682654.0  15389.3      6.2          for C in h.separate(P, only_vertices=True):
    15    316386  147388270.0    465.8     49.0              for L in all_lambdas(h.E, k):
    16    315549   66205869.0    209.8     22.0                  B = set.intersection(C, L)
    17    315549   68322904.0    216.5     22.7                  if len(B) > 1 and B not in softk:
    18       243      39620.0    163.0      0.0                      softk.append(B)
    19         1        545.0    545.0      0.0      return softk

In [8]:
%lprun -f computesoftkBlocks computesoftkBlocks(h,3)

Timer unit: 1e-09 s

Total time: 0.228984 s
File: /tmp/ipykernel_337936/3625937043.py
Function: computesoftkBlocks at line 30

Line #      Hits         Time  Per Hit   % Time  Line Contents
    30                                           def computesoftkBlocks(h, k):
    31         1       1237.0   1237.0      0.0      out = list()
    32         1  218210593.0    2e+08     95.3      listOfLists = map(partial(bag_to_blocks,h),computesoftk(h,k))
    33       244   10573823.0  43335.3      4.6      for ll in listOfLists:
    34      1081     119775.0    110.8      0.1          for l in ll:
    35       838      78634.0     93.8      0.0              out.append(l)
    36         1        289.0    289.0      0.0      return out

In [6]:
import os


c=0
for path in glob("/home/okulmus/Documents/OldBenchmarks/benchmark/hyperbench/*"):
    if os.path.isdir(path):
        continue
    h = HyperGraph.fromHyperbench(path)
    if len(h.E) <= 33:
        c += 1
c

1583

In [6]:
from ctdcheck import CTDCheck
from hypergraph import HyperGraph

h = HyperGraph.fromHyperbench("/home/okulmus/Documents/OldBenchmarks/benchmark/hyperbench/1.dtl")

ctd = CTDCheck(h)

blocks = computesoftkBlocks(h,3)

for b in blocks:
    ctd.addBlock(b)

In [16]:


%aimport ctdcheck
%aimport hypergraph


AssertionError: 